<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/CrossTab_Olap_Cube_Postgres_Superstore_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

For the corresponding exercise with MySQL see this [notebook](https://github.com/Praxis-QR/RDWH/blob/main/Cross_Tab_with_OLAP_Cube_SQL_Superstore.ipynb)

#Set Up

In [1]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
!sudo -u postgres psql -V
!service postgresql start





psql (PostgreSQL) 12.14 (Ubuntu 12.14-0ubuntu0.20.04.1)
 * Starting PostgreSQL 12 database server
   ...done.


In [2]:
# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
#
# Setup a database with name `sampledb` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS praxisdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE praxisdb;'

ALTER ROLE
NOTICE:  database "praxisdb" does not exist, skipping
DROP DATABASE
CREATE DATABASE


In [3]:
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

localhost:5432:praxisdb:postgres:pass


In [4]:
#!psql -h localhost -p 5432 -Upostgres -W -dpraxisdb -c \
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c \
"CREATE TABLE IF NOT EXISTS ss_order (\
RowID smallint,\
OrderID char(14),OrderDate date,\
ShipDate date,ShipMode varchar(16),\
CustomerID char(8),CustomerName varchar(30),Segment varchar(20),\
Country varchar(30),City varchar(30),State varchar(30),PostalCode char(5),Region varchar(15) ,\
ProductID varchar(20), Category varchar(40), SubCategory varchar(40), ProductName varchar(200), \
Sales decimal(8,2), Quantity smallint, Discount decimal(4,2), Profit decimal(8,2) \
); \
"

CREATE TABLE


In [5]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c '\d+ ss_order'

                                            Table "public.ss_order"
    Column    |          Type          | Collation | Nullable | Default | Storage  | Stats target | Description 
--------------+------------------------+-----------+----------+---------+----------+--------------+-------------
 rowid        | smallint               |           |          |         | plain    |              | 
 orderid      | character(14)          |           |          |         | extended |              | 
 orderdate    | date                   |           |          |         | plain    |              | 
 shipdate     | date                   |           |          |         | plain    |              | 
 shipmode     | character varying(16)  |           |          |         | extended |              | 
 customerid   | character(8)           |           |          |         | extended |              | 
 customername | character varying(30)  |           |          |         | extended |              | 

In [6]:
#!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'
#!gdown https://drive.google.com/uc?id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
!wget -q -O SS_Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'

In [7]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "set datestyle to PostgreSQL,European; show datestyle"

   DateStyle   
---------------
 Postgres, DMY
(1 row)



In [8]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "set datestyle to PostgreSQL,European;COPY ss_order FROM '/content/SS_Orders.csv' DELIMITER ',' CSV HEADER;"

COPY 9994


In [9]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c 'select * from ss_order limit 10'
#the date data should have loaded correctly

 rowid |    orderid     | orderdate  |  shipdate  |    shipmode    | customerid |  customername   |  segment  |    country    |      city       |   state    | postalcode | region |    productid    |    category     | subcategory |                           productname                            | sales  | quantity | discount | profit  
-------+----------------+------------+------------+----------------+------------+-----------------+-----------+---------------+-----------------+------------+------------+--------+-----------------+-----------------+-------------+------------------------------------------------------------------+--------+----------+----------+---------
     1 | CA-2016-152156 | 2016-11-08 | 2016-11-11 | Second Class   | CG-12520   | Claire Gute     | Consumer  | United States | Henderson       | Kentucky   | 42420      | South  | FUR-BO-10001798 | Furniture       | Bookcases   | Bush Somerset Collection Bookcase                                | 261.96 |        2 |     0.

#Group By

In [10]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT segment, region, category, SUM (sales) sales \
FROM ss_order \
GROUP BY segment, region, category; \
"

   segment   | region  |    category     |   sales   
-------------+---------+-----------------+-----------
 Corporate   | South   | Office Supplies |  45930.19
 Consumer    | South   | Technology      |  65276.15
 Corporate   | East    | Technology      |  69725.58
 Corporate   | East    | Office Supplies |  66474.79
 Consumer    | East    | Furniture       | 114211.77
 Consumer    | West    | Furniture       | 119808.16
 Home Office | West    | Furniture       |  49724.57
 Corporate   | South   | Technology      |  46310.75
 Home Office | East    | Furniture       |  29870.32
 Consumer    | Central | Office Supplies |  93111.40
 Home Office | Central | Office Supplies |  32777.22
 Consumer    | South   | Furniture       |  70800.22
 Consumer    | South   | Office Supplies |  59504.55
 Corporate   | South   | Furniture       |  29645.01
 Consumer    | East    | Office Supplies | 101255.11
 Consumer    | West    | Office Supplies | 110080.90
 Consumer    | Central | Technology      |  

In [11]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT segment, region, SUM (sales) sales \
FROM ss_order \
GROUP BY segment, region; \
"

   segment   | region  |   sales   
-------------+---------+-----------
 Home Office | Central |  91212.66
 Corporate   | West    | 225855.25
 Home Office | South   |  74255.03
 Consumer    | Central | 252031.43
 Home Office | West    | 136721.86
 Corporate   | South   | 121885.95
 Consumer    | South   | 195580.92
 Corporate   | Central | 157995.79
 Consumer    | West    | 362880.82
 Consumer    | East    | 350908.17
 Corporate   | East    | 200409.45
 Home Office | East    | 127463.74
(12 rows)



In [12]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT segment, SUM (sales) sales \
FROM ss_order \
GROUP BY segment; \
"

   segment   |   sales    
-------------+------------
 Consumer    | 1161401.34
 Corporate   |  706146.44
 Home Office |  429653.29
(3 rows)



In [13]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT  SUM (sales) sales \
FROM ss_order  \
"

   sales    
------------
 2297201.07
(1 row)



#Grouping Sets

In [14]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT category, region,segment,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region,segment), \
        (category,region), \
        (category, segment), \
        (segment, region), \
        (category), \
        (region), \
        (segment), \
        () \
    ); \
"

    category     | region  |   segment   |  qty  
-----------------+---------+-------------+-------
                 |         |             | 37873
 Furniture       | East    | Corporate   |   723
 Furniture       | West    | Home Office |   476
 Office Supplies | South   | Corporate   |  1281
 Technology      | South   | Consumer    |   584
 Furniture       | South   | Corporate   |   415
 Technology      | South   | Corporate   |   320
 Furniture       | South   | Home Office |   181
 Furniture       | Central | Consumer    |   952
 Office Supplies | Central | Consumer    |  2777
 Office Supplies | West    | Home Office |  1301
 Office Supplies | Central | Corporate   |  1605
 Office Supplies | East    | Home Office |  1182
 Office Supplies | East    | Consumer    |  3245
 Technology      | East    | Corporate   |   600
 Technology      | West    | Consumer    |  1235
 Furniture       | East    | Consumer    |  1114
 Office Supplies | West    | Corporate   |  2097
 Furniture       |

In [15]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT category, region,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region), \
        (category), \
        (region), \
        () \
    ); \
"

    category     | region  |  qty  
-----------------+---------+-------
                 |         | 37873
 Furniture       | West    |  2696
 Office Supplies | South   |  3800
 Furniture       | East    |  2214
 Office Supplies | Central |  5409
 Technology      | East    |  1942
 Technology      | Central |  1544
 Office Supplies | East    |  6462
 Technology      | West    |  2335
 Technology      | South   |  1118
 Office Supplies | West    |  7235
 Furniture       | Central |  1827
 Furniture       | South   |  1291
 Furniture       |         |  8028
 Office Supplies |         | 22906
 Technology      |         |  6939
                 | East    | 10618
                 | South   |  6209
                 | West    | 12266
                 | Central |  8780
(20 rows)



#Roll Up

In [16]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT category, SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (category); \
"

    category     |  sum  
-----------------+-------
                 | 37873
 Furniture       |  8028
 Office Supplies | 22906
 Technology      |  6939
(4 rows)



In [17]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT COALESCE(category, 'All category') AS category, \
 SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (category); \
"

    category     |  sum  
-----------------+-------
 All category    | 37873
 Furniture       |  8028
 Office Supplies | 22906
 Technology      |  6939
(4 rows)



##Multi Column

In [18]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT category, segment, SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (category , segment); \
"

    category     |   segment   |  sum  
-----------------+-------------+-------
                 |             | 37873
 Office Supplies | Corporate   |  7018
 Furniture       | Corporate   |  2495
 Office Supplies | Home Office |  4130
 Technology      | Consumer    |  3597
 Technology      | Corporate   |  2095
 Office Supplies | Consumer    | 11758
 Technology      | Home Office |  1247
 Furniture       | Home Office |  1367
 Furniture       | Consumer    |  4166
 Furniture       |             |  8028
 Office Supplies |             | 22906
 Technology      |             |  6939
(13 rows)



In [19]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT category, segment, SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (segment , category); \
"

    category     |   segment   |  sum  
-----------------+-------------+-------
                 |             | 37873
 Office Supplies | Home Office |  4130
 Office Supplies | Consumer    | 11758
 Technology      | Corporate   |  2095
 Office Supplies | Corporate   |  7018
 Furniture       | Home Office |  1367
 Technology      | Home Office |  1247
 Furniture       | Corporate   |  2495
 Furniture       | Consumer    |  4166
 Technology      | Consumer    |  3597
                 | Consumer    | 19521
                 | Corporate   | 11608
                 | Home Office |  6744
(13 rows)



In [20]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (segment , category); \
"

    category     |   segment   | quantity 
-----------------+-------------+----------
 All category    | All segment |    37873
 Office Supplies | Home Office |     4130
 Furniture       | Corporate   |     2495
 Office Supplies | Consumer    |    11758
 Furniture       | Home Office |     1367
 Technology      | Consumer    |     3597
 Office Supplies | Corporate   |     7018
 Technology      | Corporate   |     2095
 Technology      | Home Office |     1247
 Furniture       | Consumer    |     4166
 All category    | Consumer    |    19521
 All category    | Corporate   |    11608
 All category    | Home Office |     6744
(13 rows)



## Partial Rollup

In [21]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY segment, ROLLUP (category); \
"

    category     |   segment   | quantity 
-----------------+-------------+----------
 Office Supplies | Home Office |     4130
 Furniture       | Corporate   |     2495
 Office Supplies | Consumer    |    11758
 Furniture       | Home Office |     1367
 Technology      | Consumer    |     3597
 Office Supplies | Corporate   |     7018
 Technology      | Corporate   |     2095
 Technology      | Home Office |     1247
 Furniture       | Consumer    |     4166
 All category    | Consumer    |    19521
 All category    | Corporate   |    11608
 All category    | Home Office |     6744
(12 rows)



In [22]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY category, ROLLUP (segment); \
"

    category     |   segment   | quantity 
-----------------+-------------+----------
 Furniture       | Corporate   |     2495
 Furniture       | Home Office |     1367
 Technology      | Consumer    |     3597
 Office Supplies | Corporate   |     7018
 Office Supplies | Consumer    |    11758
 Technology      | Corporate   |     2095
 Technology      | Home Office |     1247
 Office Supplies | Home Office |     4130
 Furniture       | Consumer    |     4166
 Furniture       | All segment |     8028
 Office Supplies | All segment |    22906
 Technology      | All segment |     6939
(12 rows)



#Cube

In [23]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT COALESCE(category, 'All category') category, SUM(quantity) quantity \
FROM ss_order \
GROUP BY CUBE(category) \
ORDER BY category;  \
"

    category     | quantity 
-----------------+----------
 All category    |    37873
 Furniture       |     8028
 Office Supplies |    22906
 Technology      |     6939
(4 rows)



In [24]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT \
COALESCE(category, 'All category') category, \
COALESCE(segment, 'All segment') segment, \
SUM(quantity) quantity \
FROM ss_order \
GROUP BY CUBE(category, segment) \
ORDER BY category,segment;  \
"

    category     |   segment   | quantity 
-----------------+-------------+----------
 All category    | All segment |    37873
 All category    | Consumer    |    19521
 All category    | Corporate   |    11608
 All category    | Home Office |     6744
 Furniture       | All segment |     8028
 Furniture       | Consumer    |     4166
 Furniture       | Corporate   |     2495
 Furniture       | Home Office |     1367
 Office Supplies | All segment |    22906
 Office Supplies | Consumer    |    11758
 Office Supplies | Corporate   |     7018
 Office Supplies | Home Office |     4130
 Technology      | All segment |     6939
 Technology      | Consumer    |     3597
 Technology      | Corporate   |     2095
 Technology      | Home Office |     1247
(16 rows)



In [25]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
SELECT \
COALESCE(category, 'All category') category, \
COALESCE(segment, 'All segment') segment, \
COALESCE(region, 'All region') region, \
SUM(quantity) quantity \
FROM ss_order \
GROUP BY CUBE(category, segment, region) \
ORDER BY category,segment, region;  \
"

    category     |   segment   |   region   | quantity 
-----------------+-------------+------------+----------
 All category    | All segment | All region |    37873
 All category    | All segment | Central    |     8780
 All category    | All segment | East       |    10618
 All category    | All segment | South      |     6209
 All category    | All segment | West       |    12266
 All category    | Consumer    | All region |    19521
 All category    | Consumer    | Central    |     4519
 All category    | Consumer    | East       |     5347
 All category    | Consumer    | South      |     3178
 All category    | Consumer    | West       |     6477
 All category    | Corporate   | All region |    11608
 All category    | Corporate   | Central    |     2604
 All category    | Corporate   | East       |     3358
 All category    | Corporate   | South      |     2016
 All category    | Corporate   | West       |     3630
 All category    | Home Office | All region |     6744
 All cat

#Cross Tab

##Level 1

In [27]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
    select Category, \
    sum(case when (Region = 'Central') then quantity else 0 end) as Central, \
    sum(case when (Region = 'East') then quantity else 0 end) as East, \
    sum(case when (Region = 'West') then quantity else 0 end) as West, \
    sum(case when (Region = 'South') then quantity else 0 end) as South \
    from ss_order \
    group by Category"

    category     | central | east | west | south 
-----------------+---------+------+------+-------
 Furniture       |    1827 | 2214 | 2696 |  1291
 Office Supplies |    5409 | 6462 | 7235 |  3800
 Technology      |    1544 | 1942 | 2335 |  1118
(3 rows)



## Level 2

In [29]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
  select \
  COALESCE(Category, 'All Category') AS Category, \
  COALESCE(Region, 'All Region') AS Region, \
  Sum(quantity) Quantity \
  from ss_order \
  group by ROLLUP (Category,Region)  \
"

    category     |   region   | quantity 
-----------------+------------+----------
 All Category    | All Region |    37873
 Technology      | West       |     2335
 Technology      | South      |     1118
 Office Supplies | South      |     3800
 Office Supplies | West       |     7235
 Furniture       | Central    |     1827
 Office Supplies | East       |     6462
 Furniture       | East       |     2214
 Office Supplies | Central    |     5409
 Furniture       | South      |     1291
 Technology      | East       |     1942
 Furniture       | West       |     2696
 Technology      | Central    |     1544
 Furniture       | All Region |     8028
 Office Supplies | All Region |    22906
 Technology      | All Region |     6939
(16 rows)



##Level 3

In [30]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c " \
SELECT \
COALESCE(Category, 'Totals') AS Dim, \
sums.Central, sums.East, sums.West, sums.South, \
sums.Central + sums.East + sums.West + sums.South  Totals \
FROM ( \
  select Category, \
    sum(case when (Region = 'Central') then quantity else 0 end) as Central, \
    sum(case when (Region = 'East') then quantity else 0 end) as East, \
    sum(case when (Region = 'West') then quantity else 0 end) as West, \
    sum(case when (Region = 'South') then quantity else 0 end) as South \
    from ss_order \
    group by Rollup (Category) \
) AS sums \
"

       dim       | central | east  | west  | south | totals 
-----------------+---------+-------+-------+-------+--------
 Totals          |    8780 | 10618 | 12266 |  6209 |  37873
 Furniture       |    1827 |  2214 |  2696 |  1291 |   8028
 Office Supplies |    5409 |  6462 |  7235 |  3800 |  22906
 Technology      |    1544 |  1942 |  2335 |  1118 |   6939
(4 rows)



In [31]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Calcutta')))

Last tested   2023-03-29 08:15:25.870166+05:30


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)